# 環境準備

## 既出の準備
- 以下の手順で環境を構築する。  
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?OSS%E3%81%AELLM
  - WSL2にOllamaをインストールし
  - Ollamaを起動してLlama3を実行させておく。
  - WSL2のNotebookからWebAPI経由でLLMを呼び出す。

## 追加の準備
- 必要に応じ、requestsをプロキシに振らないようにする。
- !pip install できなくなるので proxies引数で無効化してもよい。

### プロキシ

```Shell
%env HTTP_PROXY=
%env HTTPS_PROXY=
```

### LLMを選択

In [1]:
llm = "llama3"
#llm = "phi3"

### バージョン確認

In [2]:
!pip list

Package                      Version
---------------------------- ---------------
absl-py                      2.1.0
altair                       5.4.1
annotated-types              0.7.0
anyio                        4.6.2.post1
argon2-cffi                  23.1.0
argon2-cffi-bindings         21.2.0
arrow                        1.3.0
asttokens                    2.4.1
astunparse                   1.6.3
async-lru                    2.0.4
attrs                        24.2.0
babel                        2.16.0
beautifulsoup4               4.12.3
bleach                       6.1.0
blinker                      1.4
cachetools                   5.5.0
certifi                      2024.8.30
cffi                         1.17.1
charset-normalizer           3.4.0
click                        8.1.7
comm                         0.2.2
command-not-found            0.3
contourpy                    1.3.0
cryptography                 3.4.8
cycler                       0.12.1
dbus-python                  1

# 疎通確認

## HTTPClient
最も単純な、HTTPClient系ライブラリを使用した呼び出し。

### Generate
次単語予測モード

In [3]:
import requests
url = "http://localhost:11434/api/generate"
payload = {
    "model": f'{llm}',
    "prompt": "why is the sky blue?",
    "stream": False
}

response = requests.post(
    url, json=payload,
    proxies={"http": None, "https": None} #プロキシを無効にするときは必要
)
print(response.json()["response"])

The color of the sky is a fascinating topic that has puzzled humans for centuries. Here's the simple explanation:

**Short answer:** The sky appears blue because of a phenomenon called Rayleigh scattering, where shorter (blue) wavelengths of light are scattered more than longer (red) wavelengths by the tiny molecules of gases in the atmosphere.

**Longer explanation:**

When sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2). These molecules scatter the light in all directions, but they scatter shorter wavelengths (like blue and violet) more than longer wavelengths (like red and orange).

Here's what happens:

1. **Sunlight enters the Earth's atmosphere**: The sun emits a broad spectrum of electromagnetic radiation, including visible light, ultraviolet (UV), and infrared (IR) radiation.
2. **Light encounters atmospheric molecules**: As sunlight travels through the atmosphere, it encounters tiny molecules of gases like nitroge

### Chat
チャット・モード

In [4]:
import requests
url = "http://localhost:11434/api/chat"
payload = {
    "model": f'{llm}',
    "messages": [
        {"role": "user", "content": "why is the sky blue?"}
    ],
    "stream": False
}

response = requests.post(
    url, json=payload,
    proxies={"http": None, "https": None} #プロキシを無効にするときは必要
)
print(response.json()["message"]["content"])

The sky appears blue because of a phenomenon called Rayleigh scattering, which is the scattering of light by small particles or molecules in the atmosphere. Here's a simplified explanation:

1. **Sunlight enters the Earth's atmosphere**: When sunlight from the sun enters the Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2).
2. **Light is scattered**: These molecules scatter the light in all directions, but they scatter shorter wavelengths (like blue and violet) more than longer wavelengths (like red and orange). This is because the smaller molecules are more effective at scattering shorter wavelengths.
3. **Blue light reaches our eyes**: As a result of this scattering, the blue light is dispersed throughout the atmosphere, giving the sky its blue appearance.

There are several factors that contribute to why the sky appears blue:

* **Scattering by molecules**: The molecules in the atmosphere scatter light, which is what gives the sky its c

## Ollama
Ollamaライブラリを使用した呼び出し。

### ライブラリのインストール

In [ ]:
!pip install ollama

### ライブラリで呼び出し

#### Generate
次単語予測モード

In [5]:
import ollama
response = ollama.generate(
    model=f'{llm}',
    prompt='Why is the sky blue?',
)

print(response['response'])

The sky appears blue to us because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh. Here's why:

**What happens:**

When sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2). These molecules are much smaller than the wavelength of light, so they scatter the light in all directions.

**The scattering process:**

Light with shorter wavelengths (like blue and violet) is scattered more easily by the tiny molecules in the atmosphere. This is known as Rayleigh scattering. The longer wavelengths of light (like red and orange) travel more directly to our eyes, while the shorter wavelengths are dispersed in all directions.

**Why blue light scatters more:**

The reason why blue light scatters more than other colors is due to its smaller wavelength compared to the molecules in the atmosphere. As a result, blue light is scattered in a way that illuminates the entire sky, making it appear blue.



#### Chat
チャット・モード

In [6]:
import ollama
response = ollama.chat(
    model=f'{llm}',
    messages=[{
        'role': 'user',
        'content': 'Why is the sky blue?',
    }])
print(response['message']['content'])

The sky appears blue to us because of a phenomenon called Rayleigh scattering, which is named after the British physicist Lord Rayleigh. Here's what happens:

**Short-wavelength light scatters more**

When sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2). These molecules are much smaller than the wavelength of light. As a result, shorter wavelengths of light, like blue and violet, are scattered more than longer wavelengths, like red and orange.

**Why is blue scattering important?**

Blue light scatters because it has a shorter wavelength than the other colors of sunlight. This means that there's less time for the shorter blue wavelengths to scatter before they're absorbed by these tiny molecules in the atmosphere. As a result, more of the blue light is reflected back into space.

**The color we see**

As the scattered blue light is reflected back towards us, our eyes perceive it as blue light. This is why the sky appears b

# OpenAI
OpenAIライブラリを使用した呼び出し。

### ライブラリのインストール

In [ ]:
!pip install openai

### ライブラリで呼び出し

#### Generate
次単語予測モード

In [7]:
from openai import OpenAI
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ASCII code', #ダミーでOK
)

completion = client.completions.create(
    prompt='Why is the sky blue?',
    model=f'{llm}',
)

print(completion.choices[0].text)

The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh. He discovered in 1871 that the shorter (blue) wavelengths of light are scattered more than the longer (red) wavelengths when they interact with tiny molecules of gases in the atmosphere.

Here's a simplified explanation:

1. **Sunlight and Atmosphere**: When the sun shines, it emits white light, which is made up of different colors, including blue, red, green, and others.
2. **Particle Interactions**: As sunlight travels through the Earth's atmosphere, it encounters tiny molecules of gases like nitrogen (N2) and oxygen (O2). These gas molecules are in constant motion, moving randomly in any direction.
3. **Scattering Process**: When sunlight interactions with these gas molecules, the shorter blue wavelengths (around 400-450 nanometers) are scattered more than the longer red wavelengths (around 650-700 nanometers). This scattering effect is more pronounced in the atm

#### Chat
チャット・モード

In [8]:
from openai import OpenAI
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ASCII code', #ダミーでOK
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            'role': 'user',
            'content': 'Why is the sky blue?',
        }
    ],
    model=f'{llm}',
)

print(chat_completion.choices[0].message.content)

The sky appears blue because of a phenomenon called Rayleigh scattering, which occurs when sunlight interacts with tiny molecules of gases in the Earth's atmosphere.

Here's what happens:

1. **Sunlight enters the atmosphere**: When the sun shines through the atmosphere, its light is made up of a spectrum of colors, ranging from red to violet.
2. **Molecules scatter the light**: The tiny molecules of gases like nitrogen (N2) and oxygen (O2) in the atmosphere scatter the light in all directions. These molecules are much smaller than the wavelength of visible light, so they scatter the shorter wavelengths (like blue and violet) more than the longer wavelengths (like red and orange).
3. **Blue light is scattered more**: As a result, the blue light is dispersed in every direction, giving the sky its characteristic blue color.

This scattering effect is stronger when the sun is high in the sky due to:

* The amount of sunlight available
* The density of atmospheric particles

And it's why t

# langchain

In [ ]:
!pip install langchain langchain_core langchain-ollama

# Chatアプリ

## Console & HTTPClient
- 最も単純な、HTTPClient系ライブラリを使用したConsoleChatアプリ
- 以下のコードを修正したもの。  
https://github.com/ollama/ollama/tree/main/examples/python-simplechat

In [ ]:
import json
import requests

def chat(messages):
    r = requests.post(
        "http://localhost:11434/api/chat",
        json={"model": llm, "messages": messages, "stream": True},
        # proxies={"http": None, "https": None} #プロキシを無効にするときは必要
    )
    r.raise_for_status()
    output = ""

    for line in r.iter_lines():
        
        body = json.loads(line)
        
        if "error" in body:
            raise Exception(body["error"])
            
        if body.get("done") is False:
            message = body.get("message", "")
            content = message.get("content", "")
            output += content
            # the response streams one token at a time, print that as we receive it
            print(content, end="", flush=True)

        if body.get("done", False):
            message["content"] = output
            return message

def main():
    messages = []

    while True:
        user_input = input("Enter a prompt: ")
        if not user_input:
            exit()
        print()
        messages.append({"role": "user", "content": user_input})
        message = chat(messages)
        messages.append(message)
        print("\n\n")

if __name__ == "__main__":
    main()

Enter a prompt:  Why is the sky blue?



The color of the sky is a beautiful phenomenon that has puzzled people for centuries. The answer lies in a combination of atmospheric and optical factors.

**Scattering of sunlight**

When sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2). These molecules are much smaller than the wavelength of light, which is why they scatter the shorter, blue wavelengths more than the longer, red wavelengths.

This scattering effect is known as Rayleigh scattering, named after Lord Rayleigh, who first described it in the late 19th century. The shorter wavelengths are scattered in all directions by the molecules of gases in the atmosphere, while the longer wavelengths continue to travel in a straight line, reaching our eyes from about 400-700 nanometers (nm).

**Blue light dominance**

As we look up at the sky, the sun's rays have to travel through more of the Earth's atmosphere to reach our eyes. This means that the shorter blue wavelengt

## Streamlit & OpenAI
最も単純な、OpenAIライブラリを使用したConsoleChatアプリ
- Streamlitを使用してWeb化（サーバーサイド実装）
- Stlite（クライアントサイド実装）ではOpenAIが動かないらしい。

### Consoleから以下を実行

In [ ]:
!pip install streamlit

In [ ]:
!streamlit run LLM_Streamlit.py

## ブラウザからアクセスする。  
http://localhost:8501/